In [5]:
import openai
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import math

In [3]:
from dotenv import dotenv_values
config = dotenv_values(".env")
openai.api_key = config["OPENAI_API_KEY"]

In [4]:
def get_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']

In [19]:
product_url = 'https://www.amazon.com/2021-Apple-10-2-inch-iPad-Wi-Fi/dp/B09G9FPHY6'

In [7]:
from fake_useragent import UserAgent

In [12]:

def scrape_amazon_reviews(product_url, max_pages=None):
    reviews = []
    page_num = 1

    while True:
        url = f"{product_url}?ie=UTF8&reviewerType=all_reviews&pageNumber={page_num}"
        print(url)
        ua = UserAgent()
        headers = {"User-Agent": ua.random}
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")

        reviews_divs = soup.find_all("div", {"data-hook": "review"})

        if not reviews_divs:
            break

        for review in reviews_divs:
            author_elem = review.find("span", {"class": "a-profile-name"})
            rating_elem = review.find("i", {"data-hook": "review-star-rating"})
            title_elem = review.find("a", {"data-hook": "review-title"})
            date_elem = review.find("span", {"data-hook": "review-date"})
            body_elem = review.find("span", {"data-hook": "review-body"})

            data = {
                "author": author_elem.text if author_elem else None,
                "rating": rating_elem.find("span").text if rating_elem and rating_elem.find("span") else None,
                "title": title_elem.find("span").text if title_elem and title_elem.find("span") else None,
                "date": date_elem.text if date_elem else None,
                "body": body_elem.find("span").text if body_elem and body_elem.find("span") else None,
            }
            reviews.append(data)

        if max_pages and page_num >= max_pages:
            break

        page_num += 1

    return pd.DataFrame(reviews)

In [20]:

scrape_amazon_reviews(product_url)

https://www.amazon.com/2021-Apple-10-2-inch-iPad-Wi-Fi/dp/B09G9FPHY6?ie=UTF8&reviewerType=all_reviews&pageNumber=1


""


In [10]:
from langchain.vectorstores import chroma
from langchain.llms import OpenAIChat
from langchain.text_splitter import CharacterTextSplitter
from langchain import OpenAI, VectorDBQA, OpenAI
from langchain.document_loaders import DirectoryLoader
import langchain